In [4]:
%matplotlib inline

import json
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.manifold import MDS, TSNE, LocallyLinearEmbedding, Isomap
from MulticoreTSNE import MulticoreTSNE
from sklearn.decomposition import TruncatedSVD, PCA, KernelPCA, SparsePCA
import umap
import pandas as pd
from gb_writer import GlyphboardWriter
# from sklearn.model_selection import GridSearchCV
from typing import Any
import keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
import spacy
from spacy.lang.de.stop_words import STOP_WORDS


nlp = spacy.load('de')

In [5]:
def getTestData():
    with open("test_data.json", "r") as read_file:
        test_data = json.load(read_file)
    test_texts = []
    test_labels = []
    for doc in test_data:
    #     # only use confident labels
         if (doc["features"]["1"]["4"] > 0.5):
            test_labels.append(1)
            test_texts.append(doc["values"]["7"])
         else:
             test_labels.append(0)
             test_texts.append(doc["values"]["7"])
    return pd.DataFrame({'text': test_texts, 'label': test_labels})

def train(train_data, test_data, algo: Any) -> dict:
    # clean_data = [preprocessText(text) for text in data]
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        # ('nlp', preprocessText())
        # ('clf', LogisticRegression()),
        # ('clf', MultinomialNB()),
        ('clf', algo),
    ])
    text_clf.fit(train_data.text, train_data.label)
    # text_clf.fit(clean_data, labels)
    predicted = text_clf.predict(test_data.text)
    # print(dataframe.prediction.value_counts())
    # print(metrics.classification_report(test_labels, predicted))
    addHistory(metrics.f1_score(test_data.label, predicted))
    result = {
        # 'prediction': predicted.toList(),
        # 'accuracy': metrics.accuracy_score(test_labels, predicted),
        'precision': metrics.precision_score(test_data.label, predicted),
        'recall': metrics.recall_score(test_data.label, predicted),
        'f1': metrics.f1_score(test_data.label, predicted),
        'f1_history': getHistory()
    }
    return result

def loadData():
    return pd.read_csv('data.csv', sep=";", encoding="utf8")

def getHistory():
    history = []
    with open(
            "metrics.csv", "r", encoding="utf8") as file:
        reader = csv.reader(file, delimiter=';')
        for line in reader:
            history.append(line[0])
        file.close()
    return history

def addHistory(metrics):
    with open(
            "metrics.csv", "a",  newline="", encoding="utf8") as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow([str(metrics)])
        file.close()

In [6]:
def preprocessText(text: str) -> str:
    # print('Original: ', text)
    doc = nlp(text)
    # Remove Stop Words and get Lemmas
    return ' '.join([token.text for token in doc if not token.is_stop])
    # for word in doc:
    #     if word.is_stop == True:
    #         print('Stop %s', word)
    # print(word.lemma_)# getTestData()

In [7]:
def applyDR(tfidf, labels):
    # mds = MDS(n_components=2, random_state=1).fit_transform(tfidf.toarray())
    lsi = TruncatedSVD(n_components=100, random_state=1).fit_transform(tfidf.toarray())
    # mds = MDS(n_components=2, random_state=1).fit_transform(lsi)
    labels = labels.reshape(len(labels), 1)
    with_labels = np.hstack((lsi,labels))
    with_labels.shape
    mds = MDS(n_components=2, random_state=1).fit_transform(with_labels)
#     mds = TSNE(n_components=2, random_state=1).fit_transform(with_labels)
    df = pd.DataFrame(columns=['x', 'y'])
    df['x'] = mds[:, 0]
    df['y'] = mds[:, 1]
    print(df)
    return df

In [8]:
data = getTestData()
vec = TfidfVectorizer()
tfidf = vec.fit_transform(data.text)

In [9]:
# lsi = TruncatedSVD(n_components=2, random_state=1).fit_transform(tfidf.toarray())
# lsi = PCA(n_components=10, random_state=1).fit_transform(tfidf.toarray())

labels = np.asarray(data.label)
labels = labels.reshape(len(labels), 1)
# with_labels = np.hstack((lsi,labels))
with_labels = np.hstack((tfidf.toarray(),labels))


# m = Sequential()
# m.add(Dense(512,  activation='elu', input_shape=(101,)))
# m.add(Dense(128,  activation='elu'))
# m.add(Dense(2,    activation='linear', name="bottleneck"))
# m.add(Dense(128,  activation='elu'))
# m.add(Dense(512,  activation='elu'))
# m.add(Dense(101,  activation='sigmoid'))
# m.compile(loss='mean_squared_error', optimizer = Adam())
# history = m.fit(with_labels, with_labels, batch_size=128, epochs=5, verbose=1)

# encoder = Model(m.input, m.get_layer('bottleneck').output)
# lsi_with_labels = encoder.predict(with_labels)

# lsi_with_labels = PCA(n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = TruncatedSVD(n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = Isomap(n_components=2).fit_transform(with_labels)
# lsi_with_labels = MulticoreTSNE(n_jobs=1, n_components=2, random_state=1).fit_transform(with_labels)
# lsi_with_labels = LocallyLinearEmbedding(n_jobs=4,n_components=2, random_state=1).fit_transform(with_labels)
lsi_with_labels = umap.UMAP(random_state=1).fit_transform(with_labels)
df = pd.DataFrame(columns=['x', 'y'])
df['x'] = lsi_with_labels[:, 0]
df['y'] = lsi_with_labels[:, 1]
    
    
df *= 1
# DR.x *= 500
# DR.y *= 500

# print(df)

writer = GlyphboardWriter('test_name')

# DR *= 2
print('Writing positions...')    
writer.write_position(df, 'lsi')
del df
print('done')

Writing positions...
done


In [10]:
MNB = MultinomialNB()
train_data = pd.read_csv(
        'gb_training.csv', delimiter=';', encoding="utf8")
# train(train_data, data[201:], MNB)
data.loc[:800, 'label'] = 0.5
test_data = data[800+1:]
test_data

,label,text
801,0.0,"Messe the wedding showroom ""Kissed by the sun"""
802,1.0,"Sommerakademie 2014 ""Mystik und Alltag im int..."
803,0.0,Die Gans für 4 Personen zum Martinsfest Eine k...
804,1.0,Weiberfasching mit GraceComany Weiberfasching ...
805,0.0,Gästeführer Gartenerlebnis Bayern in Bayreuth ...
806,1.0,Jubiläumskonzerte im Schloss - 10 Jahre Berlin...
807,0.0,Seit 1992 wird das Mittelalterfest auf Schloss...
808,1.0,Eisdisko Eisdisko auf dem Konzertplatz. Da wer...
809,1.0,Wohnzimmermusik - Andrew Medwed Andrew Medwed ...
810,0.0,Die Verwandlung Eine Tanzproduktion nach der g...


In [11]:
data = loadData()
data.loc[data['label'] != 0.5]

,id,label,peer_label,score,text
407,408,0.0,0.0000,1.0000,Kommende Veranstaltungen Juni Juli 2013 GRILL ...
531,532,0.0,0.3125,0.6875,Schlachtplatte - Die Jahresendabrechnung Jetzt...
632,633,0.0,0.0000,1.0000,2. Spieltag TSV 1860 München - RB Leipzig e.V.


In [12]:
texts = []
labels = []
ids = []
selection_score = []
peer_labels = []
with open("test_data.json", "r") as read_file:
    test_data = json.load(read_file)



for doc in test_data:
    ids.append(doc['id'])
    texts.append(doc["values"]["7"])
    selection_score.append(1-doc["features"]["1"]["4"])
    peer_labels.append(doc["features"]["1"]["4"])
    # labels.append(doc["features"]["1"]["4"])
    if (doc["features"]["1"]["4"] > 0.5):
        labels.append(1)
    else:
        labels.append(0)

df = pd.DataFrame({
    'id': ids,
    'text': texts,
    'label': labels,
    'score': selection_score,
    'peer_label': peer_labels
})

# unlabeled = df[:200]    
test_data = df[800+1:]
test_data.to_csv('test_data.csv', sep=";", encoding="utf8", index=False)

df.loc[:, 'label'] = 0.5
df.to_csv('data.csv', sep=";", encoding="utf8", index=False)

In [13]:
data = loadData()
clean_data = [preprocessText(text) for text in data.text]
clean_data

['Silvester 2.014 AAAAAAlllttterrr Schwedeee .... Watt rast Zeit ... Und Zack fast Jahr 2014 vorbei . \r\n Und Jahr versuchen Hopfengarten Silvester Party Beine stellen . Spielregeln jedes Jahr . \r\n 20 € Mindestverzehr pro Person ( schaffen sein;- ) ) Mitbringbuffet beisteuern . \r\n Einlass 19:30 Uhr \r\n Die Listen Eintrittskarten liegen nächste Woche Hopfengarten bereit . \r\n Für Infos einfach Traumhaften Mädels Laden fragen:- ) \r\n Infos 0178 188 5153 \r\n In Sinne ... schönes Restjahr Tage ..',
 'Live Concert : Trabant Echo TRABANT ECHO / HYPER DUO feat . Cyrill Lim : HYPER FUZZ \r\n\r\n Trabant Echo Hyper Fuzz kreuzen experimentelle Popmusik , Neue Musik Rock . Mit Musik bringen Ensembles Durchlässigkeit Grenzen populärer avantgardistischer Ästhetik Geltung , Performance elektronisch groovy , anarchisch verspielt . \r\n Trabant Echos orchestrale Soundtexturen vereinen Noise , psychodelischen Kraut experimentellen Bass . Hyper Duo bringt Cyrill Lim Intensität Jazz-Rock Formati